In [6]:
import os
import pathlib
import re
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from dash.dependencies import Input, Output, State
import cufflinks as cf
import plotly
import json
import geopandas as gpd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import requests
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Initialize app

app = dash.Dash(
    __name__,
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)
app.title = "San Diego Police Calls Exploration"
server = app.server

In [8]:
database = "sdpd"
user = 'guest'
password = 'sandiego'
host = 'sdpd.chck20ykciaw.us-west-2.rds.amazonaws.com'

engine = create_engine(f"mysql://{user}:{password}@{host}:3306/{database}")
connection = engine.connect()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'sdpd.chck20ykciaw.us-west-2.rds.amazonaws.com' ([Errno 8] nodename nor servname provided, or not known)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [308]:
# Loading data for calls by beats
callsbybeats = connection.execute(
    """
    SELECT 
        *
    FROM
        call_per_beat
    """)
column_names = callsbybeats.keys()
rows = callsbybeats.fetchall()
callsbybeats_df = pd.DataFrame(rows, columns=column_names)

In [309]:
# Loading data for calls by beats
calltypessperzipcode = connection.execute(
    """
    SELECT 
        *
    FROM
        call_type_per_zipcode
    """)
column_names = calltypessperzipcode.keys()
rows = calltypessperzipcode.fetchall()
calltypessperzipcode_df = pd.DataFrame(rows, columns=column_names)

In [310]:
# Loading data for calls by type
callsbycalltype = connection.execute(
    """
    SELECT 
        *
    FROM
        calls_calltype
    """)
column_names = callsbycalltype.keys()
rows = callsbycalltype.fetchall()
callsbycalltype_df = pd.DataFrame(rows, columns=column_names)

In [311]:
# Loading data for calls by beats
callsbydispositioncode = connection.execute(
    """
    SELECT 
        *
    FROM
        calls_dispo
    """)
column_names = callsbydispositioncode.keys()
rows = callsbydispositioncode.fetchall()
callsbydispositioncode_df = pd.DataFrame(rows, columns=column_names)

In [312]:
# Loading data for calls by beats
callsperdaybyreasondispositionbeat = connection.execute(
    """
    SELECT 
        *
    FROM
        calls_per_year_per_type_disposition_beat
    """)
column_names = callsperdaybyreasondispositionbeat.keys()
rows = callsperdaybyreasondispositionbeat.fetchall()
callsperdaybyreasondispositionbeat_df = pd.DataFrame(rows, columns=column_names)

In [313]:
# Loading data for calls by zipcode
callsperzipcode = connection.execute(
    """
    SELECT 
        *
    FROM
        calls_per_zipCode
    """)
column_names = callsperzipcode.keys()
rows = callsperzipcode.fetchall()
callsperzipcode_df = pd.DataFrame(rows, columns=column_names)
callsperzipcode_df = callsperzipcode_df.sort_values(by='IncidentWeek', ascending=False)

In [314]:
# Loading data for calls by zipcode
callsperzipcodelatlong = connection.execute(
    """
    SELECT 
        *
    FROM
        calls_per_zipCode_lat_long
    """)
column_names = callsperzipcodelatlong.keys()
rows = callsperzipcodelatlong.fetchall()
callsperzipcodelatlong_df = pd.DataFrame(rows, columns=column_names)
callsperzipcodelatlong_df = callsperzipcodelatlong_df.sort_values(by='NumberOfCalls', 
                                                                  ascending=False)

In [315]:
zip_code_csv = pd.read_csv('zip_lat_long.csv')
zip_code_csv = zip_code_csv.rename(columns={'ZIP': 'ZIPCODE'})

In [316]:
zipcode_merged_df = pd.merge(callsperzipcode_df, 
                             zip_code_csv, 
                             left_on='Zipcode', 
                             right_on='ZIPCODE')

zipcode_merged_df = zipcode_merged_df.sort_values(by='IncidentWeek', ascending=False)
zipcode_merged_df = zipcode_merged_df.drop('ZIPCODE', axis=1)

In [317]:
# Get the current week number
current_week = pd.Timestamp.now().week

# # Get the week number 10 weeks ago
if current_week < 10:
    variable_weeks_ago = (pd.Timestamp.now() - pd.Timedelta(weeks=current_week-1)).week
    weeks = list(range(variable_weeks_ago, current_week + 1))
else:
    ten_weeks_ago = (pd.Timestamp.now() - pd.Timedelta(weeks=10)).week
    weeks = list(range(ten_weeks_ago, current_week + 1))


WEEKS = zipcode_merged_df1.IncidentWeek.unique()

BINS = [
    "1-50",
    "51-100",
    "101-200",
    "201-300",
    "301-400",
    "401-500",
    "501-1000",
    "1001-2000",
    "2001-5000",
]


In [318]:
# Load zip code boundaries GeoJSON file
zip_codes = gpd.read_file('https://raw.githubusercontent.com/zalvatore/Police_Calls_for_Service_San_Diego/main/Dash/Zip%20Codes.geojson')

zip_codes['geometry'] = zip_codes['geometry'].apply(lambda x: x[0] if type(x) == 'MultiPolygon' and len(x) > 1 else x)

# Change format of zipcodes 'zip' feature so that it matches data types
zip_codes['zip'] = zip_codes['zip'].tolist()
zip_codes['zip'] = [int(x) for x in zip_codes['zip']]

# Merge zip code boundaries data with call count data based on zip code
zipcode_merged_df1 = pd.merge(zip_codes, zipcode_merged_df.astype({'Zipcode': 'int', 'NumberOfCalls': 'int'}), left_on='zip', right_on='Zipcode')
zipcode_merged_df1 = zipcode_merged_df1.drop(zipcode_merged_df1[zipcode_merged_df1['IncidentYear'] == 2018].index)
zipcode_merged_df1.head()

# Define bins and bin labels for call count data
BIN_VALUES = [0, 50, 100, 200, 300, 400, 500, 1000, 2000, 5000]

# Loop through years and bins to create GeoJSON feature collections
Weeks = zipcode_merged_df1.IncidentWeek.unique()

for week in Weeks:
    week_folder = f"Week_{week}"
    if os.path.isdir(week_folder):
        if len(os.listdir(week_folder)) == 0:
            for bin_idx in range(len(BIN_VALUES) - 1):
                min_val = BIN_VALUES[bin_idx]
                max_val = BIN_VALUES[bin_idx + 1]

                # Filter the merged data to only include rows within the current bin and week
                df_subset = zipcode_merged_df1[
                    (zipcode_merged_df1["IncidentWeek"] == week) &
                    (zipcode_merged_df1["NumberOfCalls"] >= min_val) &
                    (zipcode_merged_df1["NumberOfCalls"] < max_val)
                ]

                # Create a new GeoJSON feature collection for the current bin and week
                features = []

                # Loop through the filtered data and create a GeoJSON feature for each zip code boundary
                for index, row in df_subset.iterrows():
                    # Convert the geometry to GeoJSON
                    geojson_geometry = row['geometry'].__geo_interface__
                    feature = {
                        "type": "Feature",
                        "geometry": geojson_geometry,
                        "properties": {
                            "id": int(row["Zipcode"]),
                            "call_count": int(row["NumberOfCalls"]),
                            "bin_label": BINS[bin_idx],
                            "week": int(week)
                        }
                    }
                    features.append(feature)

                # Define the GeoJSON FeatureCollection and features list
                feature_collection = {
                    "type": "FeatureCollection",
                    "features": features
                }

                # Write the GeoJSON to a file
                filename = f"{week_folder}/{min_val+1}-{max_val}.geojson"
                with open(filename, "w") as f:
                    json.dump(feature_collection, f)
        else:
            print(f"Skipping {week_folder} as it already contains files")
    else:
        os.mkdir(week_folder)
        for bin_idx in range(len(BIN_VALUES) - 1):
            min_val = BIN_VALUES[bin_idx]
            max_val = BIN_VALUES[bin_idx + 1]

            # Filter the merged data to only include rows within the current bin and week
            df_subset = zipcode_merged_df1[
                (zipcode_merged_df1["IncidentWeek"] == week) &
                (zipcode_merged_df1["NumberOfCalls"] >= min_val) &
                (zipcode_merged_df1["NumberOfCalls"] < max_val)
            ]

            # Create a new GeoJSON feature collection for the current bin and week
            features = []

            # Loop through the filtered data and create a GeoJSON feature for each zip code boundary
            for index, row in df_subset.iterrows():
                # Convert the geometry to GeoJSON
                geojson_geometry = row['geometry'].__geo_interface__
                feature = {
                    "type": "Feature",
                    "geometry": geojson_geometry,
                    "properties": {
                        "id": int(row["Zipcode"]),
                        "call_count": int(row["NumberOfCalls"]),
                        "bin_label": BINS[bin_idx],
                        "week": int(week)
                    }
                }
                features.append(feature)

            # Define the GeoJSON FeatureCollection and features list
            feature_collection = {
                "type": "FeatureCollection",
                "features": features
            }

            # Write the GeoJSON to a file
            filename = f"{week_folder}/{min_val+1}-{max_val}.geojson"
            with open(filename, "w") as f:
                json.dump(feature_collection, f)

Skipping Week_7 as it already contains files
Skipping Week_6 as it already contains files
Skipping Week_5 as it already contains files
Skipping Week_4 as it already contains files
Skipping Week_3 as it already contains files
Skipping Week_2 as it already contains files
Skipping Week_8 as it already contains files


In [319]:
# To load the colors that will be associated with each bin in the "BINS" list on the map
DEFAULT_COLORSCALE = [
    "#f2fffb",
    "#98ffe0",
    "#6df0c8",
    "#69e7c0",
    "#31c194",
    "#25a27b",
    "#1e906d",
    "#11684d",
    "#10523e",
]

DEFAULT_OPACITY = 0.8

# To load the default map from Mapbox
mapbox_access_token = "pk.eyJ1Ijoia2JhdW0yMTUiLCJhIjoiY2xlY2J0enhiMDA4NjNvb2E3d25lMHBhNCJ9.2zuE3J3ZrfpcqNQ92gooCA"
mapbox_style = "mapbox://styles/kbaum215/cledpq199000f01qryhbewf24"

In [320]:
# App layout

app.layout = html.Div(
    id="root",
    children=[
        html.Div(
            id="header",
            children=[
                html.A(
                    html.Img(
                        id="logo", 
                        src=app.get_asset_url("usd_logo.png"),
                        style={"width": "50px", 
                               "height": "50px", 
                               "margin-left": "-50px"},
                    ),
                ),
                html.H4(children="Exploration of Police Calls in San Diego County 2023"),
            ],
        ),
        html.Div(
            id="app-container",
            children=[
                html.Div(
                    id="left-column",
                    children=[
                        html.Div(
                            id="slider-container",
                            children=[
                                html.P(
                                    id="slider-text",
                                    children="Drag the slider to change the week:",
                                ),
                                dcc.Slider(
                                    id="weeks-slider",
                                    min=min(WEEKS),
                                    max=max(WEEKS),
                                    value=min(WEEKS),
                                    marks={
                                        str(week): {
                                            "label": str(week),
                                            "style": {"color": "#7fafdf"},
                                        }
                                        for week in WEEKS
                                    },
                                ),
                            ],
                        ),
                        html.Div(
                            id="heatmap-container",
                            children=[
                                html.P(
                                    "Heatmap of age adjusted mortality rates \
                            from poisonings in Week {0}".format(
                                        min(WEEKS)
                                    ),
                                    id="heatmap-title",
                                ),
                                dcc.Graph(
                                    id="county-choropleth",
                                    figure=dict(
                                        layout=dict(
                                            mapbox=dict(
                                                layers=[],
                                                accesstoken=mapbox_access_token,
                                                style=mapbox_style,
                                                center=dict(
#                                                     Setting up the latitude/longitude
#                                                     default of San Diego
                                                    lat=33.0198, lon=-116.8461
                                                ),
                                                pitch=0,
                                                zoom=8.5,
                                            ),
                                            autosize=True,
                                        ),
                                    ),
                                ),
                            ],
                        ),
                    ],
                ),
                html.Div(
                    id="graph-container",
                    children=[
                        html.P(id="chart-selector", children="Select chart:"),
                        dcc.Dropdown(
                            options=[
                                {
                                    "label": "Call Count Per Zipcode (Single Week)",
                                    "value": "call_count",
                                },
                            ],
                            value="call_count",
                            id="chart-dropdown",
                        ),
                        dcc.Graph(
                            id="selected-data",
                            figure={
                                "data": [],
                                "layout": {
                                    "title": "Call Count per Zipcode",
                                    "xaxis": {"title": "Zipcode"},
                                    "yaxis": {"title": "Number of Calls"},
                                    "paper_bgcolor": "#F4F4F8",
                                    "plot_bgcolor": "#F4F4F8",
                                    "autofill": True,
                                    "margin": dict(t=75, r=50, b=100, l=50),
                                }
                            }
                        ),
                    ],
                ),
            ],
        ),
    ],
)



In [ ]:
@app.callback(
    Output("county-choropleth", "figure"),
    [Input("weeks-slider", "value")],
    [Input("county-choropleth", "figure")],
)


def display_map(week, figure):
    cm = dict(zip(BINS, DEFAULT_COLORSCALE))

    data = [
        dict(
            lat=zipcode_merged_df["LAT"],
            lon=zipcode_merged_df["LNG"],
            text=zipcode_merged_df["Zipcode"],
            type="scattermapbox",
            hoverinfo="text",
            marker=dict(size=5, color="white", opacity=0),
        )
    ]

    annotations = [
        dict(
            showarrow=False,
            align="right",
            text="<b>Call Count Per Zip Code",
            font=dict(color="#2cfec1"),
            bgcolor="#1f2630",
            x=0.95,
            y=0.95,
        )
    ]

    for i, bin in enumerate(reversed(BINS)):
        cm_color = cm[bin]
        annotations.append(
            dict(
                arrowcolor=cm_color,
                text=bin,
                x=0.95,
                y=0.85 - (i / 20),
                ax=-60,
                ay=0,
                arrowwidth=5,
                arrowhead=0,
                bgcolor="#1f2630",
                font=dict(color="#2cfec1"),
            )
        )

    if figure is not None:
        lat = figure["layout"]["mapbox"]["center"]["lat"]
        lon = figure["layout"]["mapbox"]["center"]["lon"]
        zoom = figure["layout"]["mapbox"]["zoom"]
    else:
        lat=33.0198 
        lon=-116.8461
        zoom = 8.5

    layout = dict(
        mapbox=dict(
            layers=[],
            accesstoken=mapbox_access_token,
            style=mapbox_style,
            center=dict(lat=lat, lon=lon),
            zoom=zoom,
        ),
        hovermode="closest",
        margin=dict(r=0, l=0, t=0, b=0),
        annotations=annotations,
        dragmode="box",
    )
    
#     User must change this base_path to the directory that have downloaded the files to 
    base_path = '/Users/kevinbaum/Desktop/Police_Calls_for_Service_San_Diego/Dash/Week_'
    for bin in BINS:
        source_path = base_path + str(week) + "/" + bin + ".geojson"
        with open(source_path, "r") as f:
            geojson_data = json.load(f)
        geo_layer = dict(
            sourcetype="geojson",
            source=geojson_data,
            type="fill",
            color=cm[bin],
            opacity=DEFAULT_OPACITY,
            fill=dict(outlinecolor="#afafaf"),
        )
        layout["mapbox"]["layers"].append(geo_layer)
    
    fig = dict(data=data, layout=layout)
    return fig
    


@app.callback(Output("heatmap-title", "children"), [Input("weeks-slider", "value")])
def update_map_title(week):
    return "Heatmap of Police Calls Per San Diego Area Zipcodes \
				 in Week {0}".format(
        week
    )



@app.callback(
    Output("selected-data", "figure"),
    [
        Input("county-choropleth", "selectedData"),
        Input("chart-dropdown", "value"),
        Input("weeks-slider", "value") 
    ],
)
def display_selected_data(selectedData, chart_dropdown, week):
    if selectedData is None:
        return dict(
            data=[dict(x=0, y=0)],
            layout=dict(
                title="Click-drag on the map to select zip codes",
                paper_bgcolor="#1f2630",
                plot_bgcolor="#1f2630",
                font=dict(color="#2cfec1"),
                margin=dict(t=75, r=50, b=100, l=75),
            ),
        )

    if isinstance(selectedData, dict):
        selected_zipcodes = [point['text'] for point in selectedData['points']]
        selectedData = {'points': [{'customdata': row['Zipcode']} for row in zipcode_merged_df[(zipcode_merged_df['IncidentWeek'] == int(week)) & (zipcode_merged_df['Zipcode'].isin(selected_zipcodes))].to_dict('records')]}

    pts = selectedData["points"]
    fips = [str(pt["customdata"]) for pt in pts]

    selected_data_dict = {
    "Zipcode": [pt["customdata"] for pt in pts],
    "customdata": [pt["customdata"] for pt in pts],
    }    
    
    dff = pd.DataFrame.from_dict(selected_data_dict)
    dff = pd.merge(zipcode_merged_df, dff, on="Zipcode")
    dff['Zipcode'] = dff['Zipcode'].astype(str)
    dff = dff[dff["IncidentWeek"] == week]
    
    
    if chart_dropdown == "call_count":
        call_count_by_zipcode = dff.groupby("Zipcode")["NumberOfCalls"].sum()
        call_count_by_zipcode = call_count_by_zipcode.sort_values()

        fig = call_count_by_zipcode.iplot(
            kind="bar",
            y="NumberOfCalls",
            title="Call Count per Zipcode",
            asFigure=True,
        )

        fig_layout = fig["layout"]
        fig_data = fig["data"]

        fig_data[0]["text"] = call_count_by_zipcode.values.tolist()
        fig_data[0]["marker"]["color"] = "#2cfec1"
        fig_data[0]["marker"]["opacity"] = 1
        fig_data[0]["marker"]["line"]["width"] = 0
        fig_data[0]["textposition"] = "outside"
        fig_layout["paper_bgcolor"] = "#1f2630"
        fig_layout["plot_bgcolor"] = "#1f2630"
        fig_layout["font"]["color"] = "#2cfec1"
        fig_layout["title"]["font"]["color"] = "#2cfec1"
        fig_layout["xaxis"]["tickfont"]["color"] = "#2cfec1"
        fig_layout["yaxis"]["tickfont"]["color"] = "#2cfec1"
        fig_layout["xaxis"]["gridcolor"] = "#5b5b5b"
        fig_layout["yaxis"]["gridcolor"] = "#5b5b5b"
        fig_layout["margin"]["t"] = 75
        fig_layout["margin"]["r"] = 50
        fig_layout["margin"]["b"] = 100
        fig_layout["margin"]["l"] = 50

        return fig
    
    

if __name__ == "__main__":
    app.run_server(port=3061)
    
    

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

Dash is running on http://127.0.0.1:3061/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3061
Press CTRL+C to quit
127.0.0.1 - - [27/Feb/2023 17:00:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2023 17:00:16] "GET /assets/styles1.css?m=1676506846.0 HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2023 17:00:16] "GET /assets/styles2.css?m=1676506846.7125475 HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2023 17:00:16] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1675386571.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2023 17:00:16] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1675386571.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2023 17:00:16] "GET /_dash-component-suites/dash/deps/react@16.v2_8_1m1675386571.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2023 17:00:16] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1675386571.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2023 17:00:16] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_8_1m1675386570.min.js HTTP/1.1" 200 -
127.0.0.1 -